# Практическое задание: построение модели предсказания продаж в торговой сети

В этом практическом задании вам нужно будет построить модель, делающую долгосрочный прогноз продаж товаров в торговой сети.
Задание основано на контесте: https://www.kaggle.com/c/m5-forecasting-accuracy. 

В данном задании вы не только построите модель, но и спроектируете небольшую архитектуру, позволяющую эффективно пересчитывать предсказания модели на новых данных. 

**Внимание**. Любые скрипты, параметры запуска и т.п. можно редактировать в разумных пределах.

## Этап 1: Фильтрация дней 

Напомним, что в данной задаче вам нужно предсказать на 28 дней вперёд продажи 3049 товаров в 10 магазинах.

Скорее всего для обучения хорошей модели не потребуется информация за всю историю продаж продуктов, более того информация по достаточно давним продажам может даже вредить.
Для экономии памяти лучше всего  сразу избавиться от дней, которые не будут входить в обучение.
На этапе объединения всех табличных данных в одну, необходимо оставить только необходимое для обучения количество дней. 

Для того, чтобы сгенерировать такой массив, вам достаточно запустить скрипт `generate_melted_df`, аргумент `min_train_day` отвечает за первый день, который будет добавлен в выборку. 

In [0]:
! python feature_generation/generate_melted_df.py --min_train_day=1180

tcmalloc: large alloc 3314147328 bytes == 0x17d72a000 @  0x7fe562b69001 0x7fe5604bb765 0x7fe56051fbb0 0x7fe56051ff37 0x7fe5605b7f28 0x50a635 0x50cd96 0x509758 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509042 0x594931 0x549e5f
tcmalloc: large alloc 1301069824 bytes == 0x6e348000 @  0x7fe562b69001 0x7fe5604bb765 0x7fe56051fbb0 0x7fe56051ff37 0x7fe5605b7f28 0x50a635 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509042 0x594931 0x59fc4e
tcmalloc: large alloc 1452843008 bytes == 0xc48c4000 @  0x7fe562b671e7 0x7fe5604bb5e1 0x7fe56051fc78 0x7fe56051fd93 0x7fe5605bdea8 0x7fe5605be704 0x7fe5605be852 0x566d63 0x59fc4e 0x7fe56050b4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x588d41 0x5

In [0]:
import pandas as pd
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
df = pd.read_csv('data/melted_train.csv')

In [0]:
df = df.sort_values(by=['id', 'day_as_int'])

In [0]:
df.to_csv('data/melted_train.csv')

## Этап 2: Генерация признаков (3 балла)

Для обучения модели понадобится сгенерировать признаки, содержащие информацию о прошлых продажах товара.
Пусть $y_i$ — количество продаж каждого отдельного ряда в момент времени i.
Для удобства изложения, введём для каждого признака обозначение через результат выполнения некоторой функции.

#### Признаки, которые необходимо добавить в модель

* Количество продаж $i$ дней назад (shift_i):

    $$
    {
    f(t, i, 1, 1) = y_{t-i} 
    }
    $$


* Среднее число продаж за $j$ дней на момент времени $i$ дней назад (mean_i_j): 
    
    $$
    {
    f(t, i, j, 1) = \frac{1}{j}\sum_{k=0}^{j-1} y_{t-i-k} 
    }
    $$

* Среднее число продаж за $j$ последних дней с периодом $d$ на момент времени $i$ дней назад (mean_i_j_period_d):
    
    $$ 
    {
    f(t, i, j, d) = \frac{1}{j}\sum_{k=0}^{j-1} y_{t-i-k * d} 
    }
    $$


* Экспоненциальное скользящее среднее (c коэффициентом $\beta$) числа продаж на момент времени $i$ дней назад (exp_mean_i_beta):

    $$
    {
    g(t, i, \beta, 1) = \frac{\sum\limits_{k=0}^{t - i} y_{t-i-k} \beta^k}{\sum\limits_{k=0}^{t - i} \beta^k}
    }
    $$
    
    
* Экспоненциальное скользящее среднее (c коэффициентом $\beta$) числа продаж с периодом $d$ на момент времени $i$ дней назад (exp_mean_i_beta_period_d):
    
    $$ 
    {
    g(t, i, \beta, d) = \frac{\sum\limits_{k=0}^{[(t-i)/d]} y_{t-i-k * d} \beta^{k}}{\sum\limits_{k=0}^{[(t-i)/d]} \beta^k}
    }
    $$

Обратите внимание, что некоторые признаки являются частными случаями других признаков. Однако, возможно, функцию для их вычисления следует написать отдельно, для оптимизации времени выполнения.

Реализацию генерации признаков необходимо выполнить в модуле `generate_past_demand_features`. После реализации признаки можно сгенерировать запустив скрипт из командной строки (посмотреть пример сопоставления параметров признакм можно внутри модуля).

#### Пример

In [0]:
import json
import re


config = json.dumps({
    'shift_parameters': [7,14,21,28,35,42,49],
    'mean_parameters': [],
    'mean_period_parameters': [(7,7,7)],
    'exp_mean_parameters': [],
    'exp_mean_period_parameters': [(7,0.5,7)],
})

config = re.sub('"', '%', config)

In [119]:
! python feature_generation/generate_past_demand_features.py -c="$config"

tcmalloc: large alloc 1816059904 bytes == 0xac648000 @  0x7f5b59eb41e7 0x7f5b57a1a5e1 0x7f5b57a7ec78 0x7f5b57a7ed93 0x7f5b57b1cea8 0x7f5b57b1d704 0x7f5b57b1d852 0x566d63 0x59fc4e 0x7f5b57a6a4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x588d41 0x59fc4e 0x7f5b57a6a4ed 0x50a2bf 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50cd96 0x509758
tcmalloc: large alloc 1816059904 bytes == 0x118a36000 @  0x7f5b59eb41e7 0x7f5b57a1a5e1 0x7f5b57a7ec78 0x7f5b57a7ed93 0x7f5b57b09ed6 0x7f5b57b0a338 0x50c29e 0x507d64 0x509a90 0x50a48d 0x50cd96 0x509758 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x507d64 0x509a90 0x50a48d 0x50bfb4
tcmalloc: large alloc 1634451456 bytes == 0xac648000 @  0x7f5b59eb41e7 0x7f5b57a1a5e1 0x7f5b57a7ec78 0x7f5b57a7ef37 0x7f5b57b16f28 0x50a635 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 0x50a48d 0x50cd96 0x507d64 0x509a90 

## Этап 3: Разделение данных на обучение и контроль

Для обучения данных необходимо разбить данные на обучение, валидацию (для подбора гиперпараметров) и тест (для финальной оценки качества модели).
В качестве теста логично брать последние несколько дней, для валидации - несколько дней непосредственно перед тестом, и все остальное - для обучения.

Однако, данные отложенные для тестирования и валидации являются самыми ценными, т.к. идут непосредственно перед пресказываемыми данными. Поэтому, настроив гиперпараметры на валидации, необходимо расширить обучающие данные за счет валидации и обучить финальную модель с известными гиперпараметрами на расширенных данных. Это необходимо будет сделать перед засылкой результата модели в соревнование.

В модуле `convert_csv_to_numpy` реализовано два типа разбиения данных: train/validation/test и train/test для настройки гиперпараметров модели и для обучения финальной модели соответственно. Запустите модуль из командной строки, чтобы получить разбиения.

In [120]:
! python feature_generation/convert_csv_to_numpy.py

tcmalloc: large alloc 5084954624 bytes == 0x18356000 @  0x7f6971e4a001 0x7f696f9ae765 0x7f696fa12bb0 0x7f696fa14a4f 0x7f696faab048 0x50a635 0x50bfb4 0x509758 0x50a48d 0x50bfb4 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f6971a45b97 0x5b250a
tcmalloc: large alloc 3032096768 bytes == 0x150d9e000 @  0x7f6971e481e7 0x7f696f9ae5e1 0x7f696fa12c78 0x7f696fa12d93 0x7f696fa9ded6 0x7f696fa9e338 0x50c29e 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f6971a45b97 0x5b250a
tcmalloc: large alloc 3414564864 bytes == 0x21518c000 @  0x7f6971e481e7 0x7f696f9ae5e1 0x7f696fa12c78 0x7f696fa12d93 0x7f696fa9ded6 0x7f696fa9e338 0x50c29e 0x507d64 0x50ae13 0x634c82 0x634d37 0x6384ef 0x639091 0x4b0d00 0x7f6971a45b97 0x5b250a


## Этап 4: Пересчёт значений признаков (6 балла)

При вычислении предсказания на тестовые 28 дней вперёд возникает ситуация, когда значения некоторых признаков оказываются неизвестными. В этом случае, эти значения необходимо заполнять с помощью предсказаний модели, полученных для меньшего горизонта прогнозирования. Например, признак количество продаж один день назад, будет вычисляться как предсказание модели в предыдущий день.

В этой части для каждого из признаков необходимо реализовать функцию пересчёта значений признака с учётом поступивших новых предсказаний модели. Заметим, что некоторые признаки можно легко пересчитать только имея информацию о других признаках.

**Задача 1**. Дано значение признака $f(t,i,j, d)$ в моменты времени от $\tau$ до $\tau + k$. Какие признаки необходимо иметь в модели, чтобы за $O(1)$ можно было пересчитать $f(\tau+k+1,i,j,d)$? Приведите формулу пересчёта ниже.

**Задача 2**. Дано значение признака $g(t,i,\beta, d)$ в моменты времени от $\tau$ до $\tau + k$. Какие признаки необходимо иметь в модели, чтобы за $O(1)$ можно было пересчитать $g(\tau+k+1,i,\beta,d)$? Приведите формулу пересчёта ниже.

**Внимание**. Не забудьте учесть случаи разных $k$!

1.$f(\tau+k+1,i,j,d) = f(\tau+k+1-d,i,j,d) + \frac{1}{j}(y_{\tau+k+1-i} - y_{\tau+k+1-i-j*d})$

2.$g(\tau+k+1,i,\beta,d) = \beta g(\tau+k+1-d,i,\beta,d)  + (1-\beta)y_{\tau+k+1-i}$

Если $k < d$ нам нужно иметь признак $f(\tau+k+1-d,i,j,d)$ и $g(\tau+k+1-d,i,\beta,d)$ + слобец $y$

Реализацию пересчёта признаков необходимо выполнить в модуле `feature_recalculation`. Категориальные признаки в этом модуле пересчитывать не нужно.

## Этап 5: Построение и валидация модели

В качестве основной метрики задачи испольузется функционал WRMSSE (подробнее о нём можно узнать из соответствующей презентации к семинару). Логично, при обучении модели настраиваться именно на конкретный функционал. 

Чтобы не тратить каждый раз память на подготовку данных для подсчёта функционала по метрике, выполните скрипт `utils/generate_evaluators` и сохраните данные для эвалюаторов на диск.

In [0]:
! python utils/generate_evaluators.py

100% 42840/42840 [00:07<00:00, 5738.11it/s]
100% 42840/42840 [00:07<00:00, 5646.78it/s]


В качестве основной модели построения предсказаний вам предлагается использовать градиентный бустинг (рекомендуем использовать lightgbm для быстроты проведения экспериментов). 

Для сравнения результатов моделей вам необходимо использовать следующую схему:

1. Обучение бустинга на train выборке на функционал MSE, валидация на val выборке по функционалу WRMSE.
2. Получение предсказаний на test выборке, с использованием рекурсивного пересчёта признаков

Обратите внимание, что на шаге 1 использовать рекурсивный пересчёт не нужно, т.к. это сильно замедлит обучение (более того, для ускорения обучения рекомендуется измерять значения функционала на валидации лишь раз в $n$ итераций). Однако, на тестовых данных всё должно быть максимально приближенно к реальности, поэтому признаки необходимо пересчитывать на ходу.

**Совет**. Чтобы точно получить правильный результат на тесте, можно после считывания данных специально занулить "неизвестные" значения признаков.

## Этап 6: Эксперименты (5)

### 1 Эксперимент

Сравните по указанной выше схеме результаты две модели.

1. Модель с двумя вещественными признаками:

    $f(t, i, 1, 1)$, $i \in \{7, 28\}$


2. Модель с восемью вещественными признаками:

    $ f(t, i, 1, 1)$, $i \in [1, 7] \cup \{28\}$

Сделайте выводы о качестве работы моделей на валидации и отложенной выборке и заполните таблицу ниже:

| WRMSSE        | val без пересчёта | test без пересчёта | test с пересчётом |
|---------------|-------------------|--------------------|-------------------|
| Первая модель |    0.723          |       0.584        |    0.784          |
| Вторая модель |    0.6297         |       0.5465       |    1.44           |

Ниже пример запуска экспериментов:  

In [0]:
## ваш код для подсчёта нужных признаков

In [0]:
# здесь хранится соответствие колонок признакам
import json
import numpy as np
with open('data/final_feature_header.json') as f:
    header = json.load(f)

target_column = header.index('demand')
day_column = header.index('day_as_int')

X = {}
y = {}
X_day = {}

for val_type in ['train', 'val', 'test']:
    X[val_type] = np.load(f'data/X_{val_type}_validation.npy')
    y[val_type] = np.copy(X[val_type][:, target_column])
    X_day[val_type] = np.copy(X[val_type][:, day_column])
    # костыль, чтобы не пересчитывать большие массивы
    X[val_type][:, target_column] = 1
    X[val_type][:, day_column] = 1
    


# # может быть полезно отсортировать тестовые массивы
# # по дням внутри каждого ряда
# header_to_index = dict(zip(header, range(len(header))))
# day_index = header_to_index['day_as_int']
# item_index = header_to_index['item_id']
# store_index = header_to_index['store_id']

# for val_type in ['val', 'test']:
#     sorted_indexes = np.lexsort((
#         X[val_type][:, store_index].ravel(),
#         X[val_type][:, item_index].ravel(),
#         X_day[val_type].ravel(),
#     ))

#     X[val_type] = X[val_type][sorted_indexes]
#     X_day[val_type] = X_day[val_type][sorted_indexes]

In [0]:
import pickle
from utils.generate_evaluators import WRMSSEEvaluator
from lightgbm import LGBMRegressor
# загрузка вспомогательных сущностей

# соответствие категориальных фичей их кодам
with open('data/label_codes_dict.json', 'rb') as f:
    label_codes_dict = json.load(f)

# эвалюаторы
with open('evaluators/valid_evaluator.pkl', 'rb') as f:
    val_evaluator = pickle.load(f)
    
with open('evaluators/test_evaluator.pkl', 'rb') as f:
    test_evaluator = pickle.load(f)

Напишите класс для подачи в интерфейс LighGBM, позволяющий вычислять значение метрики, используя готовый evaluator. Не забудьте предусмотреть вычисление лишь раз в $n$ итераций.

In [0]:
class WRMSSEEvalMetric:
    def __init__(self, evaluator, ids, verbose_step=5):
        self.evaluator = evaluator
        self.verbose_step = verbose_step
        self.ids = ids
        self.current_step = 0
    
    def __call__(self, y_true, y_pred):
        if self.current_step == 0:
            buf_df = pd.DataFrame(y_pred.reshape(30490, 28))
            buf_df['id'] = self.ids
            buf_df1 = self.evaluator.valid_df.iloc[:,:1]
            buf_df2 = buf_df1.merge(buf_df, left_on='id', right_on='id')
            y = self.evaluator.score(buf_df2.iloc[:, 1:].values)
        elif self.current_step == self.verbose_step - 1:
            self.current_step = 0
        else:
            self.current_step += 1
        return 'WRMSSE', y, False

In [0]:
index_to_item_id = {value: key for key, value in label_codes_dict['item_id'].items()}
X_val_item_ids = [index_to_item_id[elem] for elem in X['val'][:, header.index('item_id')]]
index_to_store_id = {value: key for key, value in label_codes_dict['store_id'].items()}
X_val_store_ids = [index_to_store_id[elem] for elem in X['val'][:, header.index('store_id')]]
X_val_submission_index = [
            f'{item_id}_{store_id}_validation'
            for item_id, store_id in zip(X_val_item_ids, X_val_store_ids)
        ]

In [0]:
index_to_item_id = {value: key for key, value in label_codes_dict['item_id'].items()}
X_val_item_ids = [index_to_item_id[elem] for elem in X['test'][:, header.index('item_id')]]
index_to_store_id = {value: key for key, value in label_codes_dict['store_id'].items()}
X_val_store_ids = [index_to_store_id[elem] for elem in X['test'][:, header.index('store_id')]]
X_test_submission_index = [
            f'{item_id}_{store_id}_validation'
            for item_id, store_id in zip(X_val_item_ids, X_val_store_ids)
        ]

In [178]:
wrmsse_val_eval_metric = WRMSSEEvalMetric(
    val_evaluator,
    ids = X_val_submission_index[::28],
    verbose_step=10,
)
wrmsse_val_eval_metric( 0 ,y['val'] )

('WRMSSE', 0.0, False)

In [179]:
wrmsse_val_test_metric = WRMSSEEvalMetric(
    test_evaluator,
    ids = X_test_submission_index[::28],
    verbose_step=10,
)
wrmsse_val_test_metric( 0 , y['test'])

('WRMSSE', 0.0, False)

In [0]:
regressor = LGBMRegressor(
    n_estimators=300,
    learning_rate=0.1,
    objective='mse',
    metric="None"
)

wrmsse_val_eval_metric = WRMSSEEvalMetric(
    val_evaluator,
    ids = X_val_submission_index[::28],
    verbose_step=10,
)

In [0]:
mask = list(np.arange(19)) + [26,27]
X['train'] = X['train'][:, mask]

In [0]:
X['val'] = X['val'][:, mask]
X['test'] = X['test'][:, mask]

In [187]:
regressor.fit(
   X['train'], y['train'],
   eval_set=[(X['val'], y['val'])],
   eval_metric=wrmsse_val_eval_metric,
   early_stopping_rounds=30,
   verbose=10,
)

Training until validation scores don't improve for 30 rounds.
[10]	valid_0's WRMSSE: 1.22307
[20]	valid_0's WRMSSE: 0.890328
[30]	valid_0's WRMSSE: 0.768637
[40]	valid_0's WRMSSE: 0.71543
[50]	valid_0's WRMSSE: 0.683952
[60]	valid_0's WRMSSE: 0.674085
[70]	valid_0's WRMSSE: 0.671131
[80]	valid_0's WRMSSE: 0.668675
[90]	valid_0's WRMSSE: 0.674645
Early stopping, best iteration is:
[65]	valid_0's WRMSSE: 0.668319


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              metric='None', min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=300, n_jobs=-1, num_leaves=31,
              objective='mse', random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

In [0]:
with open('model4.pkl', 'wb') as fout:
    pickle.dump(regressor, fout)

In [0]:
with open('model1.pkl', 'rb') as fin:
    regressor = pickle.load(fin)

In [189]:
wrmsse_val_eval_metric( 0 , regressor.predict(X['val']))

('WRMSSE', 0.6683191050367137, False)

In [190]:
wrmsse_val_test_metric( 0 , regressor.predict(X['test']))

('WRMSSE', 0.5602416797004593, False)

In [0]:
day_column = header.index('day_as_int')

In [0]:
X['test'][:, day_column] = X_day['test']

In [0]:
a = np.load('data/X_train_evaluation.npy')
y_for_rec = a[:,6:8]
y_for_rec = y_for_rec[y_for_rec[:,1] > 1850]

In [207]:
import importlib
importlib.reload(feature_generation.feature_recalculation)

<module 'feature_generation.feature_recalculation' from '/content/feature_generation/feature_recalculation.py'>

In [0]:
import feature_generation.feature_recalculation
test_rec_pred = feature_generation.feature_recalculation.RecursivePredictor(header, regressor, label_codes_dict, y=y_for_rec)

In [0]:
y_test_pred = test_rec_pred.get_recursive_prediction(X['test'], num_col_for_rec=2, day_range=28, start=1886)

In [0]:
new_y = test_rec_pred.y[test_rec_pred.y[:,1]>=1886][:, 0]

In [211]:
wrmsse_val_test_metric( 0 , new_y)

('WRMSSE', 0.8188288213562066, False)

### 2 Эксперимент

Сравните по указанной выше схеме результаты двух моделей:

1. Модель с признаками:

    $f(t, i, 1, 1)$, $i \in \{7, 14, 21, 28, 35, 42, 49\}$
    
    
2. Модель с признаками:

    $f(t, 7, 7, 7)$, $g(t, 7, 0.5, 7)$
    
Сделайте выводы о качестве работы моделей на валидации и отложенной выборке и заполните таблицу ниже:

| WRMSSE        | val без пересчёта | test без пересчёта | test с пересчётом |
|---------------|-------------------|--------------------|-------------------|
| Первая модель |      0.6708       |     0.5574         |   0.8579          |
| Вторая модель |      0.6683       |     0.56024        |   0.818           |

## Этап 7. Получение результата для засылки в контест (1 балл)

Пришло время заслать своё решение в соревнование.
Ваше решение должно преодолеть скор бейзлайна 0.565.

Преодолевшие скор бейзлайна могут получить бонусные баллы.
Ваш бонусный балл вычисляется по следующей функции:

`
int(np.linspace(5, 0, n_students)[your_place])
`

где `n_students` — количество студентов, преодолевших бейзлайн, `your_place` — ваше место среди студентов OzonMasters.

In [0]:
X = {}
y = {}
X_day = {}

for val_type in ['train', 'test']:
    X[val_type] = np.load(f'data/X_{val_type}_evaluation.npy')
    y[val_type] = np.copy(X[val_type][:, target_column])
    X_day[val_type] = np.copy(X[val_type][:, day_column])
    # костыль, чтобы не пересчитывать большие массивы
    X[val_type][:, target_column] = 1
    X[val_type][:, day_column] = 1

In [215]:
regressor = LGBMRegressor(
    n_estimators=200,
    learning_rate=0.1,
    objective='mse',
    metric="None"
)
regressor.fit(
   X['train'], y['train'],   
)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              metric='None', min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=200, n_jobs=-1, num_leaves=31,
              objective='mse', random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=True, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

In [0]:
with open('model5.pkl', 'wb') as fout:
    pickle.dump(regressor, fout)

In [0]:
index_to_item_id = {value: key for key, value in label_codes_dict['item_id'].items()}
X_val_item_ids = [index_to_item_id[elem] for elem in X['test'][:, header.index('item_id')]]
index_to_store_id = {value: key for key, value in label_codes_dict['store_id'].items()}
X_val_store_ids = [index_to_store_id[elem] for elem in X['test'][:, header.index('store_id')]]
X_test_submission_index = [
            f'{item_id}_{store_id}_validation'
            for item_id, store_id in zip(X_val_item_ids, X_val_store_ids)
        ]

In [0]:
a = np.load('data/X_train_evaluation.npy')
y_for_rec = a[:,6:8]
y_for_rec = y_for_rec[y_for_rec[:,1] > 1850]

In [0]:
del a

In [0]:
y_1 = np.zeros((30490, 63+28, 2))
y_new = y_for_rec.reshape(30490, 63, 2)
for i in range(30490):
  y_1[i,:, 1] = np.arange(1851, 1914+28)
  y_1[i,:63, 0] = y_new[i,:,0]

In [0]:
y_1 = y_1.reshape(30490*91,2)

In [0]:
X['test'][:, day_column] = X_day['test']

In [0]:
test_rec_pred = feature_generation.feature_recalculation.RecursivePredictor(header, regressor, label_codes_dict, y=y_1)
y_test_pred = test_rec_pred.get_recursive_prediction(X['test'], num_col_for_rec=9, day_range=28)

In [0]:
sub = pd.read_csv('data/sample_submission.csv')

buf_df = pd.DataFrame(y_test_pred.reshape(30490, 28))
buf_df['id'] = X_test_submission_index[::28]
buf_df1 = sub.iloc[:,:1]
buf_df2 = buf_df1.merge(buf_df, left_on='id', right_on='id')


In [0]:
sub.iloc[:30490,1:] = buf_df2.iloc[:, 1:].values

In [0]:
sub.to_csv('data/submission.csv', index=False)